In [1]:
import torch
import sys
import os
# os.chdir(os.path.dirname(os.path.realpath(".")) + '/FidelityFusion')
os.chdir(os.path.dirname(os.path.dirname(os.path.realpath("."))))
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..')))

import GaussianProcess.kernel as kernel
from FidelityFusion_Models import *
from FidelityFusion_Models.MF_data import MultiFidelityDataManager
from DMF_acq import *
torch.manual_seed(1)

C:\Users\Fillip\AppData\Roaming\Python\Python36\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.10.2


# Define a Simple Two fidelity objective function
This is a very simple two fidelity sample, named as 'Non Linear Sin' function.
\begin{aligned}
    f_{low}(x) &= \sin(8\pi x),\\
    f_{high}(x) &= (x - \sqrt{2})f_{low}(x)^2.\\
\end{aligned}

We use this objective function to generate a small size training data set, with 8 low-fidleity samples and 4 high-fidelity training samples under the non-subset assumption.

To be mentioned, the fidelity indicator starts from 0, which means 0 represents low-fidleity and 1 stands for high-fidleity.

In [2]:
def objective_function(x, s):
    xtr = x
    if s == 0:
        Ytr = torch.sin(xtr * 8 * torch.pi)
    else:
        Ytr_l = torch.sin(xtr * 8 * torch.pi)
        Ytr = (xtr - torch.sqrt(torch.ones(xtr.shape[0])*2).reshape(-1, 1)) * torch.pow(Ytr_l, 2)
    
    return Ytr

train_xl = torch.rand(8, 1) * 10
train_xh = torch.rand(4, 1) * 10
train_yl = objective_function(train_xl, 0)
train_yh = objective_function(train_xh, 1)

data_shape = [train_yl[0].shape, train_yh[0].shape]

initial_data = [
                    {'fidelity_indicator': 0,'raw_fidelity_name': '0', 'X': train_xl, 'Y': train_yl},
                    {'fidelity_indicator': 1, 'raw_fidelity_name': '1','X': train_xh, 'Y': train_yh},
                ]

# Initialize the surrogate model: ResGP

These part we use the simplest surrogate model ResGP as our MF model and train by the initiated training dataset. It can also be replaced by other MF model in FidelityFusion Models (i.e. AR, GAR).

In [3]:
fidelity_manager = MultiFidelityDataManager(initial_data)
fi_num = len(initial_data)
kernel1 = [kernel.SquaredExponentialKernel(length_scale = 1., signal_variance = 1.) for _ in range(fi_num)]
# model = AR(fidelity_num = fi_num, kernel_list = kernel1, rho_init=1.0, if_nonsubset=True)
# train_AR(model, fidelity_manager, max_iter=100, lr_init=1e-3)
model = ResGP(fidelity_num = fi_num, kernel_list = kernel1, if_nonsubset=True)
train_ResGP(model, fidelity_manager, max_iter=100, lr_init=1e-3)

d:\ANACONDA\envs\water\lib\site-packages\torch\autograd\__init__.py:156: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:112.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag


fidelity 0, epoch 100/100, nll: 15.776910781860352
fidelity 1, epoch 100/100, nll: 7.394228935241699


# Define the mean and variance functions for acq function
The defined mean and variance functions extract the predictive mean and variance from the trained surrogate model (model) when provided with input points (x) and fidelity indicator (s). 
These functions are crucial components in the computation of acquisition functions, such as the Upper Confidence Bound (UCB), and are used to guide the selection of the next point for evaluation in the Bayesian optimization process.

In [4]:
def mean_function(x, s):
    mean, _ = model.forward(fidelity_manager, x, s)
    return mean.reshape(-1, 1)
    
def variance_function(x, s):
    _, variance = model.forward(fidelity_manager, x, s)
    return variance

# Initialize and optimize MF_acq function

In this section, we initiate a sample for MF discrete acquistion function. According to the different acq, there is a little different between the initiate parameter setting.
If want to use the KG/EI/PI acq, need specify the current best function (f_best) when initiate. But UCB do not need the f_best when initiating.

Then we use the Opitmizer to find the new_x and use different selection strategy to select next_s.

In [5]:
acq = DiscreteAcquisitionFunction(mean_function, variance_function, 2, train_xl.shape[1], torch.ones(1).reshape(-1, 1))

# Use Opitmizer to find the new_x
new_x = optimize_acq_mf(fidelity_manager, acq, 'UCB', 10, 0.01) 
# Use different selection strategy to select next_s
new_s = acq.acq_selection_fidelity(gamma=[0.1, 0.1], new_x=new_x)
print(new_x, new_s)

iter 0 x: Parameter containing:
tensor([[0.6926]], requires_grad=True) Negative Acquisition Function: -0.04308997839689255
iter 1 x: Parameter containing:
tensor([[0.7026]], requires_grad=True) Negative Acquisition Function: -0.044691674411296844
iter 2 x: Parameter containing:
tensor([[0.7126]], requires_grad=True) Negative Acquisition Function: -0.046312153339385986
iter 3 x: Parameter containing:
tensor([[0.7226]], requires_grad=True) Negative Acquisition Function: -0.04795657843351364
iter 4 x: Parameter containing:
tensor([[0.7327]], requires_grad=True) Negative Acquisition Function: -0.04963055998086929
iter 5 x: Parameter containing:
tensor([[0.7427]], requires_grad=True) Negative Acquisition Function: -0.051328860223293304
iter 6 x: Parameter containing:
tensor([[0.7528]], requires_grad=True) Negative Acquisition Function: -0.053044334053993225
iter 7 x: Parameter containing:
tensor([[0.7628]], requires_grad=True) Negative Acquisition Function: -0.054792992770671844
iter 8 x: P